# Projeto Técnicas de Programação
## Grupo 7 - Turma 1007 - ADA Santader Coders 2023 Data Science
---
## Membros
### Clara Ferreira Batista
### Laura Muglia
### Luana Ferraz
### Pedro Elias
### Rafael Couto de Oliveira
---

Nosso repositório :
https://github.com/claraferreirabatista/analise_dos_microdados_do_enem.git

---
## *Introdução*

* Motivo do trabalho: Analise dos Microdados do Enem;
* Contexto da analise do trabalho: Analise do impacto da COVID-19 nos números da região Centro-oeste referente aos anos de 2018 a 2022;
* Onde encontrar os dados : https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/enem.

### Divisão de tarefas

* Escolha do tópico: Laura Muglia
* Extração dos dados e amostras: Pedro Elias
* Carregamento, limpeza e tratativa dos dados: Rafael Couto
* Elaboração das perguntas: Luana Ferraz e Laura Muglia
* Análise exploratória: Pedro Elias, Rafael Couto
* Elaboração de gráficos: Pedro Elias, Luana Ferraz
* BugFix, reprodutibilidade e repositório: Clara Ferreira Batista
* Conclusões: Todos


---
## Importando bibliotecas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Carregamento dos dados

Puxar arquivos do Google Drive.

In [ ]:

# Importe os arquivos Parquet para DataFrames Pandas
dados_2018_co = pd.read_parquet('dados_co/ENEM_2018_CENTROOESTE.parquet')
dados_2019_co = pd.read_parquet('dados_co/ENEM_2019_CENTROOESTE.parquet')
dados_2020_co = pd.read_parquet('dados_co/ENEM_2020_CENTROOESTE.parquet')
dados_2021_co = pd.read_parquet('dados_co/ENEM_2021_CENTROOESTE.parquet')
dados_2022_co = pd.read_parquet('dados_co/ENEM_2022_CENTROOESTE.parquet')

In [ ]:
# Concatenar os DataFrames
dados = pd.concat([dados_2018_co, dados_2019_co, dados_2020_co, dados_2021_co, dados_2022_co], ignore_index=True)
dados = dados.set_index('NU_INSCRICAO')
#Configurar pandas para exibir todas as colunas
pd.set_option('display.max_columns', None)

O dataset original foi carregado com 2.018.780 linhas e 76 colunas.

 ---
# 1. Limpeza dos dados
Exclusão de linhas e colunas não interessantes, unificação e redução de colunas que representam a mesma informação, tratativa de váriaveis e export do `dataset` limpo e tratado.

## 1.1 Exclusão de Colunas
Exclusão de colunas que não se mostraram importantes para análise.

In [ ]:
#Exclusão das colunas de questões socioeconomicas não interessantes, respostas e gabaritos

#Colunas de questões
dados = dados.iloc[:,:52]

#Colunas de gabarito
col_gabaritos = ['TX_GABARITO_MT','TX_GABARITO_LC','TX_GABARITO_CH','TX_GABARITO_CN']
dados.drop(columns=col_gabaritos, inplace=True)

#Colunas de respostas
col_respostas = ['TX_RESPOSTAS_MT','TX_RESPOSTAS_LC', 'TX_RESPOSTAS_CH','TX_RESPOSTAS_CN']
dados.drop(columns=col_respostas, inplace=True)

In [ ]:
dados.info()

In [ ]:
#Exclusão de colunas não importantes para a análise

#Estado Civil
dados.drop(columns='TP_ESTADO_CIVIL', inplace=True)

#Tipo de instituição
dados.drop(columns='TP_ENSINO', inplace=True)

#Situação da escola
dados.drop(columns='TP_SIT_FUNC_ESC', inplace=True)

#codigos do municipio/estado
col_codigoslocais = ['CO_MUNICIPIO_ESC','CO_UF_ESC','CO_MUNICIPIO_PROVA','CO_UF_PROVA']
dados.drop(columns=col_codigoslocais, inplace=True)

#codigo da cor da prova
col_cores = ['CO_PROVA_CN','CO_PROVA_CH','CO_PROVA_LC','CO_PROVA_MT']
dados.drop(columns=col_cores, inplace=True)

#competencias redação
col_competencias = ['NU_NOTA_COMP1','NU_NOTA_COMP2','NU_NOTA_COMP3','NU_NOTA_COMP4','NU_NOTA_COMP5']
dados.drop(columns=col_competencias, inplace=True)


In [ ]:
#Exclusão das linhas de treineiros e da coluna
dados = dados[dados['IN_TREINEIRO'] == 0]
dados.drop(columns='IN_TREINEIRO', inplace=True)

## 1.2 Unificação de Colunas
Unificação de colunas multiplas em colunas singulares.

In [ ]:
#Unir presenças
#0	Faltou à prova
#1	Presente na prova
#2	Eliminado na prova

dados['TP_PRESENCA_MT'] =  dados['TP_PRESENCA_MT'].astype(str)
dados['TP_PRESENCA_CN'] =  dados['TP_PRESENCA_CN'].astype(str)
dados['TP_PRESENCA_CH'] =  dados['TP_PRESENCA_CH'].astype(str)
dados['TP_PRESENCA_LC'] =  dados['TP_PRESENCA_LC'].astype(str)


#Soma os TP_PRESENCA_CN, TP_PRESENCA_CH, TP_PRESENCA_LC,TP_PRESENCA_MT
dados['TP_PRESENCA'] = dados['TP_PRESENCA_CN'] + dados['TP_PRESENCA_CH'] + dados['TP_PRESENCA_LC'] + dados['TP_PRESENCA_MT']

#deletar as colunas de presença
col_presenca = ['TP_PRESENCA_MT','TP_PRESENCA_CN','TP_PRESENCA_CH','TP_PRESENCA_LC']
dados.drop(columns=col_presenca, inplace=True)

In [ ]:
#Analisando dados unidos
prop_presenca = dados['TP_PRESENCA'].value_counts(normalize=True)*100

#printando
graf1 = prop_presenca.plot(kind='bar')
plt.title('Proporções dos Valores Únicos na Coluna TP_PRESENCA')
plt.xlabel('Valor')
plt.ylabel('Proporção (%)')
#plt.xticks(rotation=0)
graf1.bar_label(graf1.containers[0], fmt='%.2f%%')
plt.tight_layout()
plt.show()

Os valores da coluna `TP_PRESENCA` serão transformados em `bool` e serão tratados da seguinte forma:
* `1 1 1 1` : Aluno Presente, 4 presenças (`TRUE`)
* `demais valores`: Aluno Ausente ou eliminado em uma ou mais provas (`FALSE`)

In [ ]:
dados['TP_PRESENCA'] = dados['TP_PRESENCA'].map({'1.01.01.01.0': True})
dados['TP_PRESENCA'] = dados['TP_PRESENCA'].fillna(False)

In [ ]:
#Nota Geral - Criar nova coluna somando as notas
dados['NU_NOTA_GERAL']=(dados['NU_NOTA_CN'] + dados['NU_NOTA_CH'] + dados['NU_NOTA_LC'] + dados['NU_NOTA_MT'])/4

## 1.3 Tratativa dos dados

Elaboração das tabelas "de-para" para valores que estão como "flag" ou valores numéricos, porém representam variaveis categóricas,substituição de flags por valores em colunas com baixa variabilidade e correção de formatos.

#### 1.3.1 Tabelas "De-para"

Elaboração das tabelas "de-para" para valores que estão como "flag" ou valores numéricos, porém representam variaveis categóricas. Usado para variáveis de alta variabilidade.

In [ ]:
#Tabela de/para: Faixa etária

# Dados para a coluna #TP_FAIXA_ETARIA
dp_idades = {
    'TP_FAIXA_ETARIA': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                         11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
    'Faixa Etária': ['Menor de 17 anos', '17 anos', '18 anos', '19 anos', '20 anos',
                     '21 anos', '22 anos', '23 anos', '24 anos', '25 anos',
                     'Entre 26 e 30 anos', 'Entre 31 e 35 anos', 'Entre 36 e 40 anos', 'Entre 41 e 45 anos',
                     'Entre 46 e 50 anos', 'Entre 51 e 55 anos', 'Entre 56 e 60 anos',
                     'Entre 61 e 65 anos', 'Entre 66 e 70 anos', 'Maior de 70 anos'],
    'Média idade': [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 28, 33, 38, 43, 48, 53, 58, 63, 68, 72]
}

dp_idades = pd.DataFrame(dp_idades).set_index('TP_FAIXA_ETARIA')

In [ ]:
#Tabela de/para: Cor/Raça

#Dados para a coluna #TP_COR_RACA
dp_raca = {
    'TP_COR_RACA': [0, 1, 2, 3, 4, 5, 6],
    'Cor/Raça': ['Não declarado', 'Branca', 'Preta', 'Parda', 'Amarela', 'Indígena', 'Não dispõe da informação']
}
dp_raca = pd.DataFrame(dp_raca).set_index('TP_COR_RACA')

In [ ]:
#Tabela de/para: Nacionalidade

#Dados para a coluna TP_NACIONALIDADE
dp_nacionalidade = {
    'TP_NACIONALIDADE': [0, 1, 2, 3, 4],
    'Nacionalidade_info': ['Não informado', 'Brasileiro(a)', 'Brasileiro(a) Naturalizado(a)',
                      'Estrangeiro(a)', 'Brasileiro(a) Nato(a), nascido(a) no exterior']
}

dp_nacionalidade = pd.DataFrame(dp_nacionalidade).set_index('TP_NACIONALIDADE')

In [ ]:
#Tabela de/para: Situação conclusão

#Dados para a coluna TP_ST_CONCLUSAO
dp_conclusao = {
    'TP_ST_CONCLUSAO': [1, 2, 3, 4],
    'Situação do Ensino Médio': ['Já concluí o Ensino Médio',
                                 'Estou cursando e concluirei o Ensino Médio no Ano da Prova',
                                 'Estou cursando e concluirei o Ensino Médio após o Ano da Prova',
                                 'Não concluí e não estou cursando o Ensino Médio']
}

dp_conclusao = pd.DataFrame(dp_conclusao).set_index('TP_ST_CONCLUSAO')

In [ ]:
#Tabela de/para: Ano Conclusão

# Dados para a coluna #TP_ANO_CONCLUIU
dp_ano_conclusao = {
    'TP_ANO_CONCLUIU': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                         11, 12, 13, 14, 15, 16],
    'Ano Concluiu': ['Não informado', '2021', '2020', '2019', '2018', '2017', '2016',
                     '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008',
                     '2007', 'Antes de 2007']
}

dp_ano_conclusao = pd.DataFrame(dp_ano_conclusao).set_index('TP_ANO_CONCLUIU')

In [ ]:
#Tabela de/para: Status Redação

# Dados para a coluna TP_STATUS_REDACAO

dp_redacao = {
    'TP_STATUS_REDACAO': [1, 2, 3, 4, 6, 7, 8, 9],
    'Status Redacao': ['Sem problemas', 'Anulada', 'Cópia Texto Motivador', 'Em Branco',
                       'Fuga ao tema', 'Não atendimento ao tipo textual', 'Texto insuficiente', 'Parte desconectada']
}

dp_redacao = pd.DataFrame(dp_redacao).set_index('TP_STATUS_REDACAO')

#### 1.3.2 Substituição de variáveis
Substituição de flags por valores em colunas com baixa variabilidade.

In [ ]:
#o Ano de 2018 possui uma diferença onde 3 signifca exterior e 4 significa escola privada
#filtra os tipo 3 do ano 2018
dados_filtrados_2018 = dados[(dados['NU_ANO'] == 2018) & (dados['TP_ESCOLA'] == 3)]
#elimina os tipo 3 do ano 2018 pelo index
dados.drop(dados_filtrados_2018.index, inplace=True)
#substitui os tipo 4 por 3
dados['TP_ESCOLA'].replace(4,3,inplace=True)

#Substituição dos valores na coluna TP_ESCOLA

dados['TP_ESCOLA'] = dados['TP_ESCOLA'].map({1:'Não Respondeu', 2:'Pública', 3:'Privada'})

In [ ]:
#Substituição dos valores na coluna TP_LINGUA

dados['TP_LINGUA'] = dados['TP_LINGUA'].map({0:'Inglês', 1:'Espanhol'})

In [ ]:
#Substituição dos valores na coluna TP_DEPENDENCIA_ADM_ESC

dados['TP_DEPENDENCIA_ADM_ESC'] = dados['TP_DEPENDENCIA_ADM_ESC'].map({1:'Federal', 2:'Estadual', 3:'Municipal',4:'Privada'})


In [ ]:
#Substituição dos valores na coluna TP_LOCALIZACAO_ESC

dados['TP_LOCALIZACAO_ESC'] = dados['TP_LOCALIZACAO_ESC'].map({1:'Urbana', 2:'Rural'})

#### 1.3.3 Tratar dados nulos
Remoção de dados ou alocação de valores para dados nulos.

In [ ]:
#excluir linhas com presença nula
dados.dropna(subset=['TP_PRESENCA'],inplace=True)

In [ ]:
#Atribuir 0 para notas nulas, NU_NOTA_CN, NU_NOTA_CH, NU_NOTA_LC,NU_NOTA_MT, NU_NOTA_REDACAO, NU_NOTA_GERAL

# Lista das colunas de notas
col_notas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO', 'NU_NOTA_GERAL']

# Preencher 0 para os valores nulos dessas colunas
dados[col_notas] = dados[col_notas].fillna(0)

In [ ]:
#atribuir 2 para redações nulas  - anulada TP_STATUS_REDACAO

dados['TP_STATUS_REDACAO'] = dados['TP_STATUS_REDACAO'].fillna(2)

#### 1.3.4 Ajustar Formatos
Corrigir tipos das colunas do dataset


In [ ]:
#converter TP_PRESENCA para bool
dados['TP_PRESENCA']=dados['TP_PRESENCA'].astype(bool)

## 1.4 Exportar Base tratada

In [ ]:
#exportar um csv unico com todos os dados tratados
dados.to_parquet('ENEM_CO_TRATADO.parquet', index=False)

Após limpeza e tratamento dos dados, a base tem xxxxx elementos e xxx colunas.

---
# 2. Análise Exploratória das variáveis
Visualizações de algumas variáveis para possíveis insights.


In [ ]:
# Importar bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
#Carregando direto os Dados Tratados:
dados_tratados = pd.read_parquet('ENEM_CO_TRATADO.parquet')


## 2.1 Análise dos inscritos
Visualização da relação de inscritos e presentes ao longo dos anos.

#### 2.1.1 Qual a faixa etária dos inscritos no ENEM entre 2018 e 2022?

O código analisa dados de faixa etária de alunos. Ele cria um DataFrame com contagem de alunos por faixa etária. Em seguida, combina esse DataFrame com um de-para das descrições das faixas etárias.

In [ ]:
#Plotar QUANTIDADE_INSCRITOS X ANO NU_ANO
inscritos = pd.DataFrame(dados_tratados['NU_ANO'].value_counts())
presente = pd.DataFrame(dados_tratados[dados_tratados['TP_PRESENCA'] == True]['NU_ANO'].value_counts())
alunos = pd.concat([inscritos.rename(columns = {'NU_ANO':'Inscritos'}),presente.rename(columns = {'NU_ANO':'Presentes'})],axis = 1)
alunos.sort_index().plot(kind='bar')
#alunos.sort_index()


Pelo gráfico acima conseguimos ver que a maior concentração dos participantes está em idade de terceiro ano do ensino médio e cursinho (de 17 a 19 anos).

#### 2.1.2 Como a faixa etária dos inscritos mudou ao longo da pandemia?

O código realiza uma análise da distribuição da quantidade de alunos por faixa etária ao longo dos anos. Inicialmente, ele mescla dados da faixa etária dos alunos com informações relacionadas a idades, usando um método de junção (merge) por correspondência na coluna 'TP_FAIXA_ETARIA'. Em seguida, calcula as contagens dessa combinação de faixa etária e ano. Esses valores são organizados em um heatmap (mapa de calor) utilizando a biblioteca Seaborn, onde as cores representam a quantidade de alunos. O

In [ ]:
#Plotar quantidades x faixas etarias por ano

Faixa_Etaria_Ano = dados_tratados.merge(dp_idades.reindex(), on = 'TP_FAIXA_ETARIA', how = 'left')
Faixa_Etaria_Ano = Faixa_Etaria_Ano[['Faixa Etária', 'NU_ANO']].value_counts()
Faixa_Etaria_Ano = Faixa_Etaria_Ano.unstack()
cmap = sns.cm.rocket_r
sns.heatmap(Faixa_Etaria_Ano, cmap = cmap)
#Faixa_Etaria_Ano.sort_index().plot.barh( stacked = True)

#printando
plt.title('Heatmap da quantidade de alunos por faixa etária por ano')
plt.xlabel('Ano')
plt.tight_layout()
plt.show()

O gráfico acima confirma que as faixas de idade de maior concentração de participantes sempre se manteve entre os participantes entre 17 e 19 anos ao longo dos anos de nossa análise. Aqui também conseguimos notar que a participação de pessoas acima de 20 anos diminuiu bastante entre 2020 e 2022, principalmente na faixa de 26 a 30 anos.

#### 2.1.3 Qual o perfil escolar dos estudantes inscritos no ENEM nos últimos 5 anos?

O código analisa dados sobre o tipo de escola em diferentes anos. Ele realiza uma contagem da frequência de tipos de escola para cada ano e organiza esses dados em um gráfico simples de barras horizontais de barras duplas.

In [ ]:
#plotar tipo de escola x anos (barras duplas)
tipo_escola = dados_tratados[['NU_ANO', 'TP_ESCOLA']].value_counts()
tipo_escola = tipo_escola.unstack()
tipo_escola.plot.barh()

#printando
plt.title('Tipo de escola por ano')
plt.xlabel('Quantidade')
plt.ylabel('Ano')
plt.tight_layout()
plt.show()

Aqui conseguimos perceber que o volume de participantes de escolas particulares e públicas não mudou muito ao longo desses 5 anos, menos em 2018 que tivemos um público de escola privada muito pequeno se comparado aos próximos anos, assim nem conseguimos visualizar o mesmo no gráfico. Já os participantes que não responderam a essa pergunta diminuiu bastante, principalmente entre 2020 e 2022.

#### 2.1.4 - Como a quantidade de ausentes mudou ao longo do tempo?

O código analisa dados de presença de alunos ao longo dos anos. Ele seleciona as colunas de "ano" e "tipo de presença" de um conjunto de dados tratados. Depois, filtra os registros em que a presença é "falsa" (ausente) e agrupa esses dados por ano, contando a quantidade de ausências em cada ano.

In [ ]:
ausentes = dados_tratados[['NU_ANO','TP_PRESENCA']]
ausentes = ausentes[ausentes['TP_PRESENCA'] == False].groupby('NU_ANO').count()

ausentes.plot.barh(legend=False)

#printando
plt.title('Quantidade de ausentes por ano')
plt.xlabel('Quantidade')
plt.ylabel('Ano')


Baseado no gráfico vemos que não existe uma tendencia clara de ausência mas que no ano de 2020 (primeiro ano da pandemia COVID-19) houve um aumento expressivo.

Baseado no gráfico vemos que não existe uma tendencia clara de ausência mas que no ano de 2020 (primeiro ano da pandemia COVID-19) houve um aumento expressivo.

#### 2.1.5 - Como a pandemia impactou na evasao à prova do enem e na quantidade de inscritos no ENEM?


O código analisa dados de participação de estudantes em anos diferentes. Dois DataFrames são criados: um com contagem de inscritos por ano e outro com contagem de presentes por ano. Esses DataFrames são combinados e um gráfico de barras é gerado para comparar inscritos e presentes ao longo dos anos.

In [ ]:
#Plotar QUANTIDADE_INSCRITOS X ANO NU_ANO
inscritos = pd.DataFrame(dados_tratados['NU_ANO'].value_counts())
presente = pd.DataFrame(dados_tratados[dados_tratados['TP_PRESENCA'] == True]['NU_ANO'].value_counts())
alunos = pd.concat([inscritos.rename(columns = {'NU_ANO':'Inscritos'}),presente.rename(columns = {'NU_ANO':'Presentes'})],axis = 1)
alunos.sort_index().plot(kind='bar')
#alunos.sort_index()

#printando
plt.title('Alunos Inscritos x Alunos Presentes')
plt.xlabel('Ano')
plt.ylabel('Quantidade')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()



Nesse gráfico acima conseguimos ver que o ano que estourou a pandemia (2020) tivemos o maior número de inscritos desde período da nossa amostra, porém também tivemos a menor conversão desses inscritos para presentes na prova, podendo isso ter sido motivado pela preocupação das pessoas em se deslocar até o local da prova pela pandemia em andamento entre outros motivos.

## 2.3 Língua

### 2.3.1 Qual foi a  preferência dos inscritos para a prova de lígua estrangeira nos últimos 5 anos?




O código examina dados sobre o tipo de língua escolhido pelos alunos em diferentes anos. Ele cria um DataFrame que conta a quantidade de escolhas de cada tipo de língua por ano. Os dados são reformulados para facilitar a visualização, gerando um gráfico de barras horizontais.

In [ ]:
#plotar quantidades linguas x anos (barras duplas)
tipo_lingua = dados_tratados[['NU_ANO', 'TP_LINGUA']].value_counts()
tipo_lingua = tipo_lingua.unstack()
tipo_lingua.plot.barh()

#printando
plt.title('Relação das linguas por ano')
plt.xlabel('Quantidade')
plt.ylabel('Ano')
plt.tight_layout()
plt.show()

Assim como observamos nos gráficos anteriores, tivemos uma redução no número de participantes do Enem nos últimos anos. Mas em relação à lingua que escolheram para fazer a prova de língua estrageira, tivemos uma queda mais significativa na escolha pela prova de espanhol do que de inglês de 2020 até 2022.

### 2.3.2 Estados que fazem divisa com países que falam espanhol tiveram maior preferêcia pela linha espanhola?





O código compara a preferência de idioma (inglês ou espanhol) em provas por estado. Ele calcula a diferença percentual entre o número de provas realizadas em espanhol e inglês em cada estado. Em seguida, cria um gráfico de barras horizontais que exibe essas diferenças. Valores de diferença percentual são adicionados ao gráfico para cada estado. O gráfico ilustra as discrepâncias na escolha de idioma entre os estados nas provas.

In [ ]:
tipo_lingua = dados_tratados[['SG_UF_PROVA','TP_LINGUA' ]].value_counts()
tipo_lingua = tipo_lingua.unstack().reset_index().set_index('SG_UF_PROVA')

#Calcular a diferença percentual
tipo_lingua['diferenca'] = 100 * (tipo_lingua['Espanhol'] - tipo_lingua['Inglês']) / tipo_lingua['Espanhol']

# Plotar o gráfico de barras horizontais
plt.figure(figsize=(10, 6))
graf34 = tipo_lingua.plot.barh()

# Adicionar os valores da coluna 'diferenca' no gráfico
for i, v in enumerate(tipo_lingua['diferenca']):
    graf34.text(v+200_000, i+0.17, f'Diferença: {v:.2f}%', color='black', fontsize=10, va='center')

#printando
plt.title('Diferença entre lingua inglesa e espanhola nos estados')
plt.xlabel('Quantidade')
plt.ylabel('Estado')

Todos os estados da região centro-oeste apresentam uma preferencia de escolha de lingua estrangeira pela língua espanhola, porém os estados onde a diferença é maior são MT (divisa com país que fala a língua espanhola) e GO (não faz dívisa com país que fala língua espanhola). Assim não é possível afirmar que a fronteira tem impacto na seleção da língua estrangeira.

## 2.4 Notas Gerais

### 2.4.1 Como foi a distribuição de notas nos últimos cinco anos?

*Não inclui nota de redação.


O código cria um conjunto de seis gráficos de histograma em uma matriz de 2x3 para visualizar as notas gerais dos alunos nos anos de 2018 a 2022. Ele itera através desses anos, filtrando os dados correspondentes a cada ano. Em seguida, cria um DataFrame com as notas gerais e plota um histograma para cada ano nos subplots da matriz.

In [ ]:
#Distribuicao normal?
#nota geral nao inclui a redacao
#Shapiro Test:
import numpy as np
from scipy.stats import shapiro
#import warnings
#warnings.filterwarnings('ignore')

filtro_notageral = dados_tratados['NU_NOTA_GERAL'] > 0
anos = [2018,2019,2020,2021,2022]
fig, axes = plt.subplots(nrows=2, ncols=3,figsize=(10, 10))
i = 0
j = 0
for ano in anos:
  filtro2 = dados_tratados['NU_ANO'] == ano
  amostra = pd.DataFrame(dados_tratados.loc[filtro_notageral & filtro2]['NU_NOTA_GERAL'])
  amostra.rename(columns={'NU_NOTA_GERAL':ano}).plot(kind = 'hist', ax=axes[i,j])

  if j <= 1 :
    j = j+1
  else:
    j = 0
    i = i+1
  #printando


#  pvalue = shapiro(amostra)[1]
#  if pvalue > 0.05:
#    print(f'Amostra de {ano} é distribuida normalmente com pvalue de {pvalue}')
#  elif pvalue < 0.05:
#    print(f'Amostra de {ano} NÃO é distribuida normalmente com pvalue de {pvalue}')



Pelos gráficos acima conseguimos ver que tivemos uma mudança de patamar de notas geraus acima de 600 (sem incluir a nota de redação) de 2020 pra frente. O volume de notas boas caiu muito, o que dá uma primeira impressão de termos resultadores piores em períodos pandêmicos e pós-pandêmicos.

### 2.4.2 Distribuição de notas dos últimos cinco anos no formato Box Plot.


O código processa dados de notas de alunos tratados. Ele seleciona as colunas de 'NU_NOTA_GERAL' e 'NU_ANO' para alunos que atendem a um certo critério. Em seguida, usando a biblioteca gráfica Plotly, ele cria um gráfico de caixa (box plot) em que os anos estão no eixo x e as notas gerais, normalizadas por 4, estão no eixo y. O gráfico exibe a distribuição das notas gerais ao longo dos anos.

In [ ]:
# plotar grafico de linha da média das notas ao longo do tempo
notas = dados_tratados.loc[filtro_notageral][['NU_NOTA_GERAL','NU_ANO']]
fig = go.Figure()
fig.add_trace(go.Box(
    x = notas['NU_ANO'],
    y = notas['NU_NOTA_GERAL']
))

#Adicionando elementos no grafico
fig.update_layout(
    title='Distribuição das Notas Gerais por Ano',
    xaxis_title='Ano',
    yaxis_title='Nota geral'

)

fig.update_traces(
    boxmean=True,  # Adiciona a linha de média dentro da caixa
    line_width=2.5,  # Espessura da linha dos box plots
)

Pelos bloxplots acima conseguimos ver que estamos com uma maior amplitude nas notas auferidas de 2020 para frente. As notas máximas e quartis superiores também se apresentam num valor acima do que vemos em 2019. A nota mínima também aumentou nesse período, contradizendo o que estavamos pensando ao ver o gráfico de cima.


### 2.4.3 As notas médias dos alunos de escolas públicas e privadas foram afetadas de forma semelhante pela pandemia?

*Não inclui nota da redação.


O código seleciona informações relevantes de alunos, incluindo ano, tipo de escola, presença e nota geral. Ele calcula a média das notas gerais para alunos presentes em cada ano e tipo de escola. Em seguida, plota gráficos de linhas sobrepostas para mostrar a evolução da média das notas ao longo dos anos, separadas por tipos de escola.

In [ ]:

presentes = dados_tratados[['NU_ANO','TP_ESCOLA','TP_PRESENCA','NU_NOTA_GERAL']]
presentes = presentes[presentes['TP_PRESENCA']==True].groupby(['NU_ANO','TP_ESCOLA']).agg({'NU_NOTA_GERAL': 'mean'})

presentes.reset_index(inplace=True)

In [ ]:
#PARA CADA TIPO DE ESCOLA PLOTAR A MEDIA AO LONGO DOS ANOS, TRES LINHAS UMA EM CIMA DA OUTRA
# Criar uma lista de tipos de escola únicos
tipos_escola = presentes['TP_ESCOLA'].unique()

# Plotar os gráficos de linha sobrepostos para cada tipo de escola
for tipo_escola in tipos_escola:
    df_tipo_escola = presentes[presentes['TP_ESCOLA'] == tipo_escola]
    plt.plot(df_tipo_escola['NU_ANO'], df_tipo_escola['NU_NOTA_GERAL'], label=f'Tipo Escola {tipo_escola}')

plt.xticks(anos)
plt.title('Média das Notas por Ano para Cada Tipo de Escola')
plt.xlabel('Ano')
plt.ylabel('Média das Notas')
plt.legend(loc=0)
plt.show()

A partir do gráfico vemos que, exceto no ano de 2018, os alunos de escola privada tem uma média de notas mais altas.

## 2.5 Notas de redação

### 2.5.1 Houve mudança significativa nas notas de redação nos útimos 5 anos?

O código processa dados de notas de estudantes. Ele seleciona as colunas de 'NU_NOTA_REDACAO' e 'NU_ANO' de um DataFrame chamado 'dados_tratados' com base em um filtro. O código calcula a média e o desvio padrão das notas por ano e armazena esses cálculos em um novo DataFrame chamado 'media'. Em seguida, ele extrai a coluna de médias de notas ('NU_NOTA_REDACAO') desse DataFrame 'media' e a representa visualmente usando um gráfico de barras horizontais, onde cada barra representa um ano, a média das notas é o valor da barra, e a barra é acompanhada por barras de erro representando o desvio padrão das notas.

In [ ]:
filtro_redacao = dados_tratados['NU_NOTA_REDACAO'] > 0
notas_redacao = dados_tratados.loc[filtro_redacao][['NU_NOTA_REDACAO','NU_ANO']]

media_redacao = notas_redacao.groupby('NU_ANO').agg([np.mean, np.std])

media_redacao2 = media_redacao['NU_NOTA_REDACAO']
media_redacao2.rename(columns={'mean':'Média Notas'}).plot(kind = 'barh', xerr = 'std',legend=False)

#printando
plt.title('Média e desvio padrão das notas da redação por ano')
plt.xlabel('Média das notas')
plt.ylabel('Ano')

###  2.5.2 Correlação entre as notas de redação e as notas nas provas de Linguagens, Códigos e suas Tecnologias

O código calcula a correção entre as notas da prova de linguagens e código e as notas da redação dos participantes. A correlação é retornada como um valor entre zero e um que expressao o nível de correlação.

In [ ]:
#plotar scatterplot notas LC (linguagens e codigos) x notas redaçaão (hipotese, quem vai bem em portugues vai bem em redação)
notas_correlacao_LC = dados_tratados.loc[filtro_notageral & filtro_redacao][['NU_NOTA_LC','NU_NOTA_REDACAO']]
notas_correlacao_LC.corr()


### 2.5.3 Correlação entre as notas nas provas objetivas e as notas de redação


O código calcula a correção entre as notas gerais e as notas da redação dos participantes. A correlação é retornada como um valor entre zero e um que expressao o nível de correlação.

In [ ]:
#plotar scatterplot notas gerais x notas redaçaão

notas_correlacao = dados_tratados.loc[filtro_notageral & filtro_redacao][['NU_NOTA_GERAL','NU_NOTA_REDACAO']]
notas_correlacao['NU_NOTA_GERAL'] = notas_correlacao['NU_NOTA_GERAL']

notas_correlacao.corr()


### 2.5.4 Distribuição das notas de redação

O código processa dados de notas de alunos tratados. Ele seleciona as colunas de 'NU_NOTA_REDACAO' e 'NU_ANO' para alunos que atendem a um certo critério. Em seguida, usando a biblioteca gráfica Plotly, ele cria um gráfico de caixa (box plot) em que os anos estão no eixo x e as notas gerais, normalizadas por 4, estão no eixo y. O gráfico exibe a distribuição das notas gerais ao longo dos anos.

In [ ]:
# plotar grafico de linha da média das notas ao longo do tempo

fig = go.Figure()
fig.add_trace(go.Box(
    x = notas_redacao['NU_ANO'],
    y = notas_redacao['NU_NOTA_REDACAO']
))

#Adicionando elementos no grafico
fig.update_layout(
    title='Distribuição das Notas das redações por Ano',
    xaxis_title='Ano',
    yaxis_title='Nota Redação'

)

fig.update_traces(
    boxmean=True,  # Adiciona a linha de média dentro da caixa
    line_width=2.5,  # Espessura da linha dos box plots
)

### 2.5.5 Houve mais dificuldade com redação durante a pandemia?

O código une dados de tipos de redação e seus status com dados tratados. Ele calcula a proporção de cada status de redação para cada ano, transforma os dados em um formato tabular e cria um gráfico de barras. O gráfico ilustra a relação entre os status das redações e os anos.

In [ ]:
#Criando variável tipo de redaçao
#plotar status redação geral (sem ser por ano, apenas tipos)
tipo_redacao = dados_tratados.merge(dp_redacao.reindex(), on = 'TP_STATUS_REDACAO', how = 'left')
tipo_redacao = tipo_redacao[['NU_ANO' , 'Status Redacao' ]].value_counts(normalize=True) * 100
tipo_redacao = tipo_redacao.unstack()

In [ ]:
# Plotar o gráfico de barras
ax = tipo_redacao.plot.bar()

# Personalizar a legenda
legend_labels = tipo_redacao.columns  # Labels da legenda (Status Redacao)

# Posicionar a legenda abaixo do gráfico, centralizado
ax.legend(legend_labels, loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=len(legend_labels))

# Resto do código de personalização
plt.title('Relação dos status das redações por ano')
plt.xlabel('Ano')
plt.ylabel('Proporção (%)')
plt.xticks(rotation=0)
plt.tight_layout()

# Mostrar o gráfico
plt.show()

## 2.6 Alunos do terceiro ano

### 2.6.1 Como as notas de redação dos terceiro anistas foi impactada pela pandemia?

O código realiza uma análise das notas de redação entre estudantes do terceiro ano do ensino médio. Ele filtra os dados para terceiro-anistas, calcula a média e o desvio padrão das notas de redação agrupadas por ano. Em seguida, renomeia colunas e compara essas médias com uma média geral de notas de redação. Um gráfico é gerado para exibir as médias das notas dos terceiro-anistas e a média geral ao longo dos anos. Intervalos de confiança são sombreados para indicar a variabilidade das médias. O objetivo é analisar como as notas de redação dos terceiro-anistas se comparam com a média geral em diferentes anos.

In [ ]:
#Entre os Terceiro Anistas:
filtro_terceiroanista = dados_tratados['TP_ST_CONCLUSAO'] == 2
terc_sit_EM = dados_tratados.loc[filtro_terceiroanista & filtro_redacao][['NU_NOTA_REDACAO','NU_ANO']]
media_terceiroanista = terc_sit_EM.groupby('NU_ANO').agg([np.mean, np.std])
media_terceiroanista2 = media_terceiroanista['NU_NOTA_REDACAO']
#renomeando colunas
media_terceiroanista2 = media_terceiroanista2.rename(columns={'mean':'mean 3a','std':'std 3a'})
media_redacao_compara = media_redacao2.rename(columns={'mean':'mean geral','std':'std geral'})

grafico_redacao = pd.concat([media_terceiroanista2,media_redacao_compara], axis = 1).reset_index()
grafico_redacao.plot(x = 'NU_ANO',y=["mean 3a", "mean geral"],  style='.-' , figsize = (15,8), xticks =grafico_redacao['NU_ANO'])
y1 = np.array(grafico_redacao['mean 3a']) - np.array(grafico_redacao['std 3a'])
y2 = np.array(grafico_redacao['mean 3a']) + np.array(grafico_redacao['std 3a'])
plt.fill_between(grafico_redacao['NU_ANO'],y1,y2,alpha=.1)
y1 = np.array(grafico_redacao['mean geral']) - np.array(grafico_redacao['std geral'])
y2 = np.array(grafico_redacao['mean geral']) + np.array(grafico_redacao['std geral'])
plt.fill_between(grafico_redacao['NU_ANO'],y1,y2,alpha=.1)




O desempenho médio dos alunos do terceiro ano na redação do ENEM tem sido melhor do que o desempenho médio de todos os participantes desde 2019. Em 2020, a diferença entre as notas médias dos alunos do terceiro ano e de todos os participantes diminuiu um pouco, mas aumentou novamente nos anos seguintes. O desvio padrão das notas dos alunos do terceiro ano também é maior do que o do público geral.

### 2.6.2 Houve alguma mudança significativa na participação de terceiro anistas diante da pandemia?


O código combina dados de inscrição com informações de conclusão do Ensino Médio. Ele cria um gráfico de barras empilhadas para representar a distribuição das situações de conclusão do Ensino Médio ao longo dos anos de inscrição.

In [ ]:
#Código
inscritos_sit_EM = dados_tratados.merge(dp_conclusao.reindex(), on = 'TP_ST_CONCLUSAO', how = 'left')
inscritos_sit_EM = inscritos_sit_EM[['NU_ANO','Situação do Ensino Médio']].value_counts()
inscritos_sit_EM.unstack().plot.bar(stacked=True, figsize = (15,8)).legend(loc=0)

#printando
plt.title('Situação do ensino médio por ano')
plt.xlabel('Ano')
plt.ylabel('Quantidade')
plt.xticks(rotation=0)


#presente = pd.DataFrame(dados_tratados[dados_tratados['TP_PRESENCA'] == True]['NU_ANO'].value_counts())
#alunos = pd.concat([inscritos.rename(columns = {'NU_ANO':'Inscritos'}),presente.rename(columns = {'NU_ANO':'Presentes'})],axis = 1)
#alunos.sort_index().plot(kind='bar')

A participação de pessoas que já concluíram o ensino médio no ENEM diminuiu significativamente após 2020.

---
# 3. Conclusão

## 3.1 Dificuldades e aprendizados


*   Pouco tempo para decisão do dataset
*   Dificuldade em trabalhar com dataset muito grande (garantir a reprodutibilidade)
* Dificuldade em trabalhar num ambiente colaborativo e versionado.



## 3.2 Pontos Positivos


*   Equipe multidisciplinar
*   Troca de experiências



## 3.3 Melhorias futuras
* Realizar testes estatisticos de normalidade
* Realizar testes estatisticos de hipótese